# Baseline models
This notebook incorporates baseline models that are used for comparison to the transformer models. It incorporates four baseline models. All models are trained for the specific prediction length of 24, 48, 96, 192.
1. Load, Scale, Split Data
2. Stationarity test
3. Naive Baseline: This model uses the value of t to predict the value at t+prediction length. The results are obtained for specific univariate rows as well as aggregates over all rows for the multivariate setting. 
4. ARIMA model: This model is a classical statistical univariate model. The results are obtained for specific univariate rows as well as aggregates over all rows for the multivariate setting. The model parameters are optimized.
5. VAR model: This model is a classical statistical multivariate model. The number of lags is optimized.
6. DLinear: The paper "Are Transformers Effective for Time Series Forecasting?" (2022) by Ailing Zeng, Muxi Chen, Lei Zhang, Qiang Xu is a highly cited and controversial paper highlighting the question whether simple linear models can outperform complex transformer models. Given the recent discussions, this model is also added as a baseline. Note this baseline model is located within the Informer_Autoformer_DLinear.ipynb  in the 04_transformer_models folder since it uses the same library as Autoformer and Informer models.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import pmdarima as pm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
import time

## 1. Load, Scale, Split Data

In [2]:
file_path = '../../01_datasets/df_all_columns.csv'
df_all_columns = pd.read_csv(file_path)

file_path = '../../01_datasets/df_most_important_columns.csv'
df_most_important_columns = pd.read_csv(file_path)

file_path = '../../01_datasets/df_only_generation_columns.csv'
df_only_generation_columns = pd.read_csv(file_path)

In [3]:
def preprocess_data(df):
    
    # Extract the date column
    dates = df['date']
    
    # Exclude the date column for scaling
    df = df.drop(columns=['date'])
    
    # Define the sizes for training, validation, and testing sets
    train_size = int(len(df) * 0.7)
    print(train_size)
    val_size = int((len(df) * 0.1)+1)
    print(val_size)
    test_size = len(df) - train_size - val_size
    
    # Split the data into training, validation, and test sets (70%,10%,20%)
    train_data = df.iloc[:train_size]
    val_data = df.iloc[train_size:train_size + val_size]
    test_data = df.iloc[train_size + val_size:]
    
    # Initialize the StandardScaler
    scaler = StandardScaler()
    
    # Fit the scaler on the training data
    scaler.fit(train_data)
    
    # Transform the datasets using the same scaler
    train_standardized = scaler.transform(train_data)
    val_standardized = scaler.transform(val_data)
    test_standardized = scaler.transform(test_data)
    
    # Create new DataFrames with standardized values, including the date column
    train_data = pd.DataFrame(train_standardized, columns=train_data.columns)
    val_data = pd.DataFrame(val_standardized, columns=df.columns)
    test_data = pd.DataFrame(test_standardized, columns=test_data.columns)
    
    # Add the date column back to the data
    train_data['date'] = dates.iloc[:train_size].values
    val_data['date'] = dates.iloc[train_size:train_size + val_size].values
    test_data['date'] = dates.iloc[train_size + val_size:].values
    
    # Merge the training and validation datasets into a single training dataset
    # This is required since the baseline models do not have a validation dataset
    # However, the scaling should be only learned on the training data to keep if fair with the transformer models
    train_data = pd.concat([train_data, val_data])
     
    # Set 'date' column as index and convert it to datetime format
    train_data['date'] = pd.to_datetime(train_data['date'])
    train_data.set_index('date', inplace=True)

    test_data['date'] = pd.to_datetime(test_data['date'])
    test_data.set_index('date', inplace=True)
 
    return train_data, test_data


In [ ]:
train_data_all_columns, test_data_all_columns = preprocess_data(df_all_columns)
train_data_most_important_columns, test_data_most_important_columns = preprocess_data(df_most_important_columns)
train_data_only_generation_columns, test_data_only_generation_columns = preprocess_data(df_only_generation_columns)

In [5]:
test_data_all_columns

,DE_load_actual_entsoe_transparency,DE_solar_capacity,DE_solar_generation_actual,DE_solar_profile,DE_wind_capacity,DE_wind_generation_actual,DE_wind_profile,DE_wind_offshore_capacity,DE_wind_offshore_generation_actual,DE_wind_offshore_profile,...,DE_amprion_solar_generation_actual,DE_amprion_wind_onshore_generation_actual,DE_tennet_load_actual_entsoe_transparency,DE_tennet_solar_generation_actual,DE_tennet_wind_generation_actual,DE_tennet_wind_offshore_generation_actual,DE_tennet_wind_onshore_generation_actual,DE_transnetbw_load_actual_entsoe_transparency,DE_transnetbw_solar_generation_actual,DE_transnetbw_wind_onshore_generation_actual
date,,,,,,,,,,,,,,,,,,,,,
2018-12-31 02:00:00,-1.635552,4.275220,-0.654636,-0.656259,2.504121,-0.435270,-0.664740,3.396873,-0.500580,-0.995570,...,-0.648871,0.057069,-1.816831,-0.648378,-0.579784,-0.514453,-0.551613,-1.733872,-0.638384,0.342018
2018-12-31 03:00:00,-1.563887,4.275220,-0.654636,-0.656259,2.504121,-0.514137,-0.724019,3.396873,-0.543178,-1.020385,...,-0.648871,0.040408,-1.767136,-0.648378,-0.663400,-0.533734,-0.651903,-1.669831,-0.638384,0.632425
2018-12-31 04:00:00,-1.473536,4.275220,-0.654636,-0.656259,2.504121,-0.581542,-0.774566,3.396873,-0.443236,-0.962643,...,-0.648871,-0.090958,-1.657685,-0.648378,-0.679094,-0.407527,-0.719757,-1.608351,-0.638384,0.569875
2018-12-31 05:00:00,-1.379805,4.275220,-0.654636,-0.656259,2.504121,-0.650691,-0.826952,3.396873,-0.395723,-0.935443,...,-0.648871,-0.153116,-1.509210,-0.648378,-0.722059,-0.310242,-0.811773,-1.582094,-0.638384,0.248194
2018-12-31 06:00:00,-1.141850,4.275220,-0.654636,-0.656259,2.504121,-0.686948,-0.854064,3.396873,-0.389989,-0.932102,...,-0.648871,-0.223605,-1.228113,-0.648378,-0.733380,-0.291836,-0.833287,-1.445685,-0.638384,0.453713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-30 19:00:00,-0.177206,5.978582,-0.654636,-0.656259,2.792954,2.760164,1.653102,4.318698,2.723772,0.588028,...,-0.648871,1.928873,-0.249151,-0.648378,2.487503,2.131526,2.394885,-0.501072,-0.638384,-0.497928
2019-12-30 20:00:00,-0.378186,5.978582,-0.654636,-0.656259,2.792954,2.769135,1.659535,4.318698,3.026055,0.741448,...,-0.648871,1.906445,-0.517443,-0.648378,2.485702,2.455809,2.270432,-0.608022,-0.638384,-0.417508
2019-12-30 21:00:00,-0.513664,5.978582,-0.654636,-0.656259,2.792954,2.650024,1.572225,4.318698,3.002298,0.729518,...,-0.648871,1.839801,-0.606772,-0.648378,2.290427,2.431269,2.028476,-0.702163,-0.638384,-0.158376


In [6]:
print(len(train_data_most_important_columns))
print(len(test_data_most_important_columns))

35035
8758


## 2. Stationarization

In [16]:
# Initialize a dictionary to store stationarity information for each column
stationarity_dict = {'Column': [], 'Is Stationary': []}

for column in train_data_all_columns.columns:
    # Check stationarity using the ADF test
    adf_test = adfuller(train_data_all_columns[column])
    is_stationary = adf_test[1] <= 0.05
    stationarity_dict['Column'].append(column)
    stationarity_dict['Is Stationary'].append(is_stationary)

# Create a DataFrame from the dictionary
stationarity_df = pd.DataFrame(stationarity_dict)

print("Stationarity Check Results:")
print(stationarity_df)

Stationarity Check Results:
                                           Column  Is Stationary
0              DE_load_actual_entsoe_transparency           True
1                               DE_solar_capacity          False
2                      DE_solar_generation_actual           True
3                                DE_solar_profile           True
4                                DE_wind_capacity          False
5                       DE_wind_generation_actual           True
6                                 DE_wind_profile           True
7                       DE_wind_offshore_capacity          False
8              DE_wind_offshore_generation_actual           True
9                        DE_wind_offshore_profile           True
10                       DE_wind_onshore_capacity          False
11              DE_wind_onshore_generation_actual           True
12                        DE_wind_onshore_profile           True
13     DE_50hertz_load_actual_entsoe_transparency           Tr

## 3. Naive Baseline

In [18]:
# Define the time shifts
time_shifts = [24, 48, 96, 192]
# Create a folder to store results
results_folder = "results_naive_predictions"
os.makedirs(results_folder, exist_ok=True)

def naive_baseline(train_data,test_data,dataset):
    # Initialize dictionaries to store MSE, MAE
    mse_dict = {}
    mae_dict = {}
    # Iterate through each time shift for the multivariate case
    for shift in time_shifts:
        mse_col = {}
        mae_col = {}
        
        # Extract the corresponding rows from the end of train_data to get test_data set predictions from the start
        train_data_subset = train_data.iloc[-shift:]
        helper_subset = pd.concat([train_data_subset,test_data])
        
        # Naive baseline model: Predict the value at t-shift to be the same as the value at t
        naive_predictions = helper_subset.shift(shift).dropna()
        actual_values = test_data

        # Iterate through each column
        for column in train_data.columns:

            # Calculate Mean Squared Error (MSE)
            mse = mean_squared_error(naive_predictions[column], actual_values[column])
            mse_col[column] = mse

            # Calculate Mean Absolute Error (MAE)
            mae = mean_absolute_error(naive_predictions[column], actual_values[column])
            mae_col[column] = mae
            
            np.save(os.path.join(results_folder, f"{dataset}_naive_predictions_shift_{shift}.npy"), naive_predictions)
            np.save(os.path.join(results_folder, f"{dataset}_actual_values_shift_{shift}.npy"), actual_values)

        # Store MSE, MAE for the current time shift
        mse_dict[shift] = mse_col
        mae_dict[shift] = mae_col
    
    # Univariate case
    if(dataset=="most_important_columns"):
        # Create a DataFrame from the list of dictionaries
        mse_dict_df = pd.DataFrame(mse_dict)
        mae_dict_df = pd.DataFrame(mae_dict)

        # Save overall results to a CSV file
        mse_dict_df_filepath = os.path.join(results_folder, f'univariate_result_most_important_columns_mse.csv')
        mse_dict_df.to_csv(mse_dict_df_filepath, index=False)
        # Save overall results to a CSV file
        mae_dict_df_filepath = os.path.join(results_folder, f'univariate_result_most_important_columns_mae.csv')
        mae_dict_df.to_csv(mae_dict_df_filepath, index=False)
    
    # Calculate the average MSE, MAE over all columns for each time shift
    average_mse = {shift: sum(mse_dict[shift].values()) / len(mse_dict[shift]) for shift in time_shifts}
    average_mae = {shift: sum(mae_dict[shift].values()) / len(mae_dict[shift]) for shift in time_shifts}

    # Print the results
    for shift in time_shifts:
        print(f"Time Shift {shift} - Average MSE: {average_mse[shift]:.2f}, Average MAE: {average_mae[shift]:.2f}")

    # Save overall MSE, MAE and shift to a CSV file
    # Create a list of dictionaries for the overall results
    overall_results = []
    for shift in time_shifts:
        overall_results.append({
            'Time Shift': shift,
            'Average MSE': average_mse[shift],
            'Average MAE': average_mae[shift]
        })

    # Create a DataFrame from the list of dictionaries
    overall_results_df = pd.DataFrame(overall_results)

    # Save overall results to a CSV file
    overall_results_filepath = os.path.join(results_folder, f'dataset_{dataset}_overall_results.csv')
    overall_results_df.to_csv(overall_results_filepath, index=False)

    print(f"Overall results saved to {overall_results_filepath}")

In [19]:
naive_baseline(train_data_all_columns, test_data_all_columns,"all_columns")
naive_baseline(train_data_most_important_columns, test_data_most_important_columns,"most_important_columns")
naive_baseline(train_data_only_generation_columns, test_data_only_generation_columns,"only_generation_columns")

Time Shift 24 - Average MSE: 1.11, Average MAE: 0.61
Time Shift 48 - Average MSE: 1.46, Average MAE: 0.73
Time Shift 96 - Average MSE: 1.72, Average MAE: 0.81
Time Shift 192 - Average MSE: 1.70, Average MAE: 0.78
Overall results saved to results_naive_predictions/dataset_all_columns_overall_results.csv
Time Shift 24 - Average MSE: 0.63, Average MAE: 0.50
Time Shift 48 - Average MSE: 1.00, Average MAE: 0.66
Time Shift 96 - Average MSE: 1.22, Average MAE: 0.75
Time Shift 192 - Average MSE: 1.12, Average MAE: 0.66
Overall results saved to results_naive_predictions/dataset_most_important_columns_overall_results.csv
Time Shift 24 - Average MSE: 1.43, Average MAE: 0.74
Time Shift 48 - Average MSE: 1.87, Average MAE: 0.89
Time Shift 96 - Average MSE: 2.19, Average MAE: 0.99
Time Shift 192 - Average MSE: 2.15, Average MAE: 0.94
Overall results saved to results_naive_predictions/dataset_only_generation_columns_overall_results.csv


In [11]:
actual_values = 'results_naive_predictions/most_important_columns_actual_values_shift_24.npy'
naive_predictions = 'results_naive_predictions/most_important_columns_naive_predictions_shift_24.npy'

## 4. ARIMA Model

In [15]:
train_data=train_data_most_important_columns
test_data=test_data_most_important_columns

# Set frequency of data
train_data.index.freq = 'H'
test_data.index.freq = 'H'

# Define different prediction lengths
prediction_lengths = [24, 48, 96, 192]

# Create a folder to store models and results
results_folder = 'arima_results'

# Create the folder if it doesn't exist
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

# Initialize lists to store overall MSE and MAE for each column
overall_mse_dict = {}
overall_mae_dict = {}

# Iterate through different prediction lengths
for prediction_length in prediction_lengths:
    for column_name in train_data.columns:
        
        print(f"Prediction Length: {prediction_length}, Column: {column_name}")
        
        # Initialize lists to store predicted and true values for the whole dataframe
        predicted_values = []
        true_values = []
        
        # Initialize lists to store predicted and true values for this column
        mse_list = []
        mae_list = []

        # Initialize a counter for iterations
        iteration = 0

        # Create a combined_data DataFrame for the current column 
        # Due to the rolling forecast that we make we need to retrain the ARIMA model everytime since ARIMA has no encoder to capture new datapoints
        # Since the ARIMA model only runs on CPU training so many models takes really long
        # Therefore, we always use the last 24 days as training values
        combined_data = train_data[column_name].copy()
        combined_data = combined_data.iloc[-576:]
        
        best_params=()
        
        while iteration <= len(test_data):
            # Calculate the end index of the current rolling window
            end_index = iteration + prediction_length

            # Extract the next prediction length steps of test_data for this column
            current_window = test_data[column_name].iloc[iteration:end_index]
            # Use PMDARIMA to auto-select the ARIMA model for this column
            model = pm.auto_arima(combined_data, seasonal=False, stepwise=True, trace=False, suppress_warnings=True)

            # Make a forecast for the current window
            forecast, conf_int = model.predict(n_periods=prediction_length, return_conf_int=True)
            
            if len(forecast) != len(current_window):
                forecast = forecast[:len(current_window)]
                
           # Calculate Mean Squared Error (MSE) for the current iteration
            mse = mean_squared_error(current_window, forecast)
            mse_list.append(mse)

            # Calculate Mean Absolute Error (MAE) for the current iteration
            mae = mean_absolute_error(current_window, forecast)
            mae_list.append(mae)

            # Append the current window to combined_data 
            combined_data = pd.concat([combined_data,current_window])
            combined_data.index.freq = 'H'
            # Ensure combined_data only contains the last 24 days to train the next model
            combined_data = combined_data.iloc[-576:]
            
            # Append predicted and true values
            predicted_values.extend(forecast)
            true_values.extend(current_window.values)
            
            # Just some information to know how far we are in the prediction process
            if iteration%768==0:
                print('This is iteration '+str(iteration))
                # Get the best model's parameters
                best_params = model.order
                print(f"Best ARIMA parameters: {best_params}")
            # Increment the iteration counter
            iteration += prediction_length

        # Calculate the average MSE and MAE for this column and prediction length
        average_mse = np.mean(mse_list)
        average_mae = np.mean(mae_list)
        print(f"Average MSE: {average_mse}, Average MAE: {average_mae}")

        # Store overall MSE and MAE for this column and prediction length in dictionaries
        overall_mse_dict[(prediction_length, column_name)] = average_mse
        overall_mae_dict[(prediction_length, column_name)] = average_mae
        
        # Save predicted and true values as numpy arrays
        predicted_values = np.array(predicted_values)
        true_values = np.array(true_values)
        np.save(os.path.join(results_folder, f'predicted_values_{column_name}__{prediction_length}.npy'), predicted_values)
        np.save(os.path.join(results_folder, f'true_values_{column_name}__{prediction_length}_.npy'), true_values)

# Save overall MSE and MAE to a CSV file
overall_results_df = pd.DataFrame({'Prediction Length': [p[0] for p in overall_mse_dict.keys()],
                                   'Column Name': [p[1] for p in overall_mse_dict.keys()],
                                   'Average Overall MSE': list(overall_mse_dict.values()),
                                   'Average Overall MAE': list(overall_mae_dict.values())})
overall_results_filepath = os.path.join(results_folder, f'overall_results.csv')
overall_results_df.to_csv(overall_results_filepath, index=False)

print("Results saved successfully.")

Prediction Length: 24, Column: DE_load_actual_entsoe_transparency
This is iteration 0
Best ARIMA parameters: (2, 1, 2)
This is iteration 768
Best ARIMA parameters: (3, 0, 4)
This is iteration 1536
Best ARIMA parameters: (2, 0, 1)
This is iteration 2304
Best ARIMA parameters: (2, 0, 1)
This is iteration 3072
Best ARIMA parameters: (4, 0, 4)
This is iteration 3840
Best ARIMA parameters: (2, 0, 1)
This is iteration 4608
Best ARIMA parameters: (4, 0, 2)
This is iteration 5376
Best ARIMA parameters: (2, 0, 1)
This is iteration 6144
Best ARIMA parameters: (5, 0, 2)
This is iteration 6912
Best ARIMA parameters: (5, 0, 3)
This is iteration 7680
Best ARIMA parameters: (2, 0, 1)
This is iteration 8448
Best ARIMA parameters: (2, 0, 1)
test
Average MSE: 0.8061958944786651, Average MAE: 0.7486365000542853
Prediction Length: 24, Column: DE_solar_generation_actual
This is iteration 0
Best ARIMA parameters: (3, 0, 1)
This is iteration 768
Best ARIMA parameters: (4, 0, 5)
This is iteration 1536
Best AR

/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(5,0,1)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "/vol/fob-vol5/nebenf22/pohc

This is iteration 6912
Best ARIMA parameters: (4, 0, 2)
This is iteration 7680
Best ARIMA parameters: (3, 0, 1)
This is iteration 8448
Best ARIMA parameters: (3, 0, 1)
test
Average MSE: 0.6273841542079568, Average MAE: 0.5662764950762543
Prediction Length: 24, Column: DE_wind_generation_actual
This is iteration 0
Best ARIMA parameters: (2, 1, 0)
This is iteration 768
Best ARIMA parameters: (1, 1, 1)
This is iteration 1536
Best ARIMA parameters: (2, 1, 0)
This is iteration 2304
Best ARIMA parameters: (2, 1, 2)
This is iteration 3072
Best ARIMA parameters: (2, 1, 1)
This is iteration 3840
Best ARIMA parameters: (5, 1, 2)
This is iteration 4608
Best ARIMA parameters: (2, 1, 2)
This is iteration 5376
Best ARIMA parameters: (2, 1, 0)
This is iteration 6144
Best ARIMA parameters: (2, 1, 0)
This is iteration 6912
Best ARIMA parameters: (2, 1, 2)
This is iteration 7680
Best ARIMA parameters: (2, 0, 3)
This is iteration 8448
Best ARIMA parameters: (1, 1, 1)
test
Average MSE: 0.6348729146816469,

/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(5,0,1)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "/vol/fob-vol5/nebenf22/pohc

This is iteration 6912
Best ARIMA parameters: (4, 0, 2)
This is iteration 7680
Best ARIMA parameters: (3, 0, 1)
This is iteration 8448
Best ARIMA parameters: (3, 0, 1)
test
Average MSE: 0.785700654057152, Average MAE: 0.6512788344617646
Prediction Length: 48, Column: DE_wind_generation_actual
This is iteration 0
Best ARIMA parameters: (2, 1, 0)
This is iteration 768
Best ARIMA parameters: (1, 1, 1)
This is iteration 1536
Best ARIMA parameters: (2, 1, 0)
This is iteration 2304
Best ARIMA parameters: (2, 1, 2)
This is iteration 3072
Best ARIMA parameters: (2, 1, 1)
This is iteration 3840
Best ARIMA parameters: (5, 1, 2)
This is iteration 4608
Best ARIMA parameters: (2, 1, 2)
This is iteration 5376
Best ARIMA parameters: (2, 1, 0)
This is iteration 6144
Best ARIMA parameters: (2, 1, 0)
This is iteration 6912
Best ARIMA parameters: (2, 1, 2)
This is iteration 7680
Best ARIMA parameters: (2, 0, 3)
This is iteration 8448
Best ARIMA parameters: (1, 1, 1)
test
Average MSE: 0.9447545527626104, 

/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(5,0,1)(0,0,0)[0]           (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "/vol/fob-vol5/nebenf22/pohchris/projectA/time/lib/python3.8/site-packages/pmdarima/arima/arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "/vol/fob-vol5/nebenf22/pohc

This is iteration 6912
Best ARIMA parameters: (4, 0, 2)
This is iteration 7680
Best ARIMA parameters: (3, 0, 1)
This is iteration 8448
Best ARIMA parameters: (3, 0, 1)
test
Average MSE: 0.9536795126391064, Average MAE: 0.7277359920057889
Prediction Length: 96, Column: DE_wind_generation_actual
This is iteration 0
Best ARIMA parameters: (2, 1, 0)
This is iteration 768
Best ARIMA parameters: (1, 1, 1)
This is iteration 1536
Best ARIMA parameters: (2, 1, 0)
This is iteration 2304
Best ARIMA parameters: (2, 1, 2)
This is iteration 3072
Best ARIMA parameters: (2, 1, 1)
This is iteration 3840
Best ARIMA parameters: (5, 1, 2)
This is iteration 4608
Best ARIMA parameters: (2, 1, 2)
This is iteration 5376
Best ARIMA parameters: (2, 1, 0)
This is iteration 6144
Best ARIMA parameters: (2, 1, 0)
This is iteration 6912
Best ARIMA parameters: (2, 1, 2)
This is iteration 7680
Best ARIMA parameters: (2, 0, 3)
This is iteration 8448
Best ARIMA parameters: (1, 1, 1)
test
Average MSE: 1.4841774916124095,

## 5. VAR model
A vectorautoregressive (VAR) model is a statistical model, that uses a specific number of up to t past values of the different time series to predict future values in a multivariate way.
1) To apply a VAR model properly the timeseries should be stationary. Therefore this characteristic is checked before.
2) Different VAR models are trained using different predicition lengths and optimizing the number of lags used.
3) Accuracy values for specific columns is calculated

In [7]:
# Define different prediction lengths
prediction_lengths = [24, 48, 96, 192]

# Define different lag orders to test
lag_orders = [1, 2, 4, 12, 24, 48]

# Create a folder to store models and results
results_folder = 'var_results'

# Create the folder if it doesn't exist
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

In [10]:
def var_model(train_data,test_data,dataset):
    
    # Create a subfolder with the name of lag order
    subfolder_name = f'{dataset}'
    subfolder_path = os.path.join(results_folder,subfolder_name)
    # Create the folder if it doesn't exist
    if not os.path.exists(subfolder_path):
        os.makedirs(subfolder_path)
        
    # Set frequency of data
    train_data.index.freq = 'H'
    test_data.index.freq = 'H'

    # Initialize lists to store MSE and MAE for each iteration
    mse_list = []
    mae_list = []

    # Initialize dictionaries to store overall MSE and MAE for each combination
    overall_mse_dict = {}
    overall_mae_dict = {}

    # Iterate through different prediction lengths and lag orders
    for prediction_length in prediction_lengths:
        # Create a subfolder with the name of lag order
        subfolder_name = f'{prediction_length}'
        final_path = os.path.join(subfolder_path,subfolder_name)
        if not os.path.exists(final_path):
            os.makedirs(final_path)
            
        for lag in lag_orders:

            print(prediction_length,lag)

            # Create a VAR model using train_data
            model = VAR(train_data)

            # Initialize lists to store predicted and true values
            predicted_values = []
            true_values = []

            # Initialize a DataFrame to keep track of combined data
            combined_data = train_data.copy()

            # Initialize a counter for iterations
            iteration = 0

            while iteration <= (len(test_data)):
                # Calculate the end index of the current rolling window
                end_index = iteration + prediction_length

                # Extract the next prediction length steps of test_data
                current_window = test_data.iloc[iteration:end_index]

                # Fit the VAR model to the combined_data
                model_fitted = model.fit(lag)

                # Make a forecast for the current window using the combined_data so far
                forecast = model_fitted.forecast(combined_data.values, steps=prediction_length)

                if len(forecast) != len(current_window):
                    forecast = forecast[:len(current_window)]

                # Calculate Mean Squared Error (MSE) for the current iteration
                mse = mean_squared_error(current_window, forecast)
                mse_list.append(mse)

                # Calculate Mean Absolute Error (MAE) for the current iteration
                mae = mean_absolute_error(current_window, forecast)
                mae_list.append(mae)

                # Add the current window to combined_data for the next iteration
                # This means that iteratively part of the test_data is added to the training_data for the next iteration 
                # This enables a fair comparison to the transformer model as the decoder of the transformer model also receives a sequence length of inputs from the test_data set to predict the next steps
                combined_data = pd.concat([combined_data,current_window])
                combined_data.index.freq = 'H'

                # Update the model with the combined_data
                model = VAR(combined_data)

                # Increment the iteration counter
                iteration += prediction_length

                # Append predicted and true values
                predicted_values.extend(forecast)
                true_values.extend(current_window.values)
                
            # Calculate the average MSE and MAE for this combination
            average_mse = np.mean(mse_list)
            average_mae = np.mean(mae_list)
            print(average_mse, average_mae)
            # Store overall MSE and MAE for this combination in dictionaries
            overall_mse_dict[(prediction_length, lag)] = average_mse
            overall_mae_dict[(prediction_length, lag)] = average_mae

            # Save the VAR model to a file
            model_filename = f'/var_{dataset}_{prediction_length}_{lag}.pkl'
            model_path = final_path + model_filename

            with open(model_path, 'wb') as model_file:
                pickle.dump(model_fitted, model_file)

            # Save predicted and true values as numpy arrays
            predicted_values = np.array(predicted_values)
            true_values = np.array(true_values)
            np.save(os.path.join(final_path, f'predicted_values_{lag}.npy'), predicted_values)
            np.save(os.path.join(final_path, f'true_values_{lag}.npy'), true_values)

    # Save overall MSE and MAE to a CSV file
    overall_results_df = pd.DataFrame({'Prediction Length': [p[0] for p in overall_mse_dict.keys()],
                                       'Lag Order': [p[1] for p in overall_mse_dict.keys()],
                                       'Average Overall MSE': list(overall_mse_dict.values()),
                                       'Average Overall MAE': list(overall_mae_dict.values())})
    overall_results_filepath = os.path.join(results_folder, f'dataset_{dataset}overall_results.csv')
    overall_results_df.to_csv(overall_results_filepath, index=False)

    print("Results saved successfully.")

In [ ]:
var_model(train_data_all_columns, test_data_all_columns,"all_columns")
var_model(train_data_most_important_columns, test_data_most_important_columns,"most_important_columns")
var_model(train_data_only_generation_columns, test_data_only_generation_columns,"only_generation_columns")

24 1
0.9858323249989994 0.6568739327511877
24 2
0.8312182800056853 0.6040087593257766
24 4
0.7633134612081417 0.579009024745949
24 12
0.7155314747643353 0.5582858053737348
24 24
0.6693295496675038 0.5320665531355984
24 48
